<a href="https://colab.research.google.com/github/Tee-A/AIRLINES-AIRCRAFTS_TAXI-OUT-TIME-PREDICTION/blob/master/Temmy_Best_DSN2020_catty.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IMporting Catboost

In [ ]:
!pip install catboost

     |████████████████████████████████| 66.1MB 57kB/s 


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from tqdm import tqdm
import math
import gc
warnings.filterwarnings('ignore')

from math import sqrt 
import lightgbm as lgb
from sklearn.metrics import mean_squared_error 
from sklearn.model_selection import KFold, train_test_split
import requests
from io import StringIO 
%matplotlib inline
pd.set_option("display.max_rows", 200)
pd.set_option("display.max_columns", 200)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.inspection import permutation_importance
from sklearn.metrics import plot_confusion_matrix, plot_roc_curve, roc_auc_score, auc, classification_report

In [ ]:
import random
seed = 4097
random.seed(seed)
np.random.seed(seed)

In [ ]:
path = '/content/drive/My Drive/dsn_2020_q'

# Reading Datasets

In [ ]:
train  = pd.read_csv(path+'/Train.csv')
test = pd.read_csv(path+'/Test.csv')
subfile = pd.read_csv(path+'/SampleSubmission.csv')

In [ ]:
train.describe()

,form_field1,form_field2,form_field3,form_field4,form_field5,form_field6,form_field7,form_field8,form_field9,form_field10,form_field11,form_field12,form_field13,form_field14,form_field15,form_field16,form_field17,form_field18,form_field19,form_field20,form_field21,form_field22,form_field23,form_field24,form_field25,form_field26,form_field27,form_field28,form_field29,form_field30,form_field31,form_field32,form_field33,form_field34,form_field35,form_field36,form_field37,form_field38,form_field39,form_field40,form_field41,form_field42,form_field43,form_field44,form_field45,form_field46,form_field48,form_field49,form_field50
count,53471.000000,52156.000000,55645.000000,55645.000000,55645.000000,4.264000e+04,5.083700e+04,4.264000e+04,4.799200e+04,5.564500e+04,2.457900e+04,4.610500e+04,5.011100e+04,5.600000e+04,3.352500e+04,42964.000000,44849.000000,45598.000000,55996.000000,55645.000000,40146.000000,35600.000000,27877.000000,42703.000000,50550.000000,48562.000000,46701.000000,55645.000000,55645.000000,30491.000000,16592.000000,50550.000000,54744.000000,55645.000000,32852.000000,54005.000000,50550.000000,55645.000000,51789.000000,12271.000000,17771.000000,54677.000000,55432.000000,50617.000000,24683.000000,40096.000000,3.511100e+04,55645.000000,4.494400e+04
mean,3491.795665,0.550737,1.052225,0.851979,1.956317,6.244479e+05,6.865210e+06,2.626690e+06,1.316002e+07,1.185585e+07,2.054715e+06,2.771113e+06,6.009944e+06,6.914875e+07,1.145041e+08,0.230425,0.265692,0.271832,0.948800,1.031593,86.500380,6628.316381,6452.875072,104.558516,2624.555935,10106.554899,6190.516948,17274.315141,16518.540695,5213.246663,10857.565755,938.345321,13.926583,0.230245,0.199866,12.859069,8.480435,8.601420,0.090849,147.797977,108.117363,0.368215,6.634511,0.563377,0.072520,0.095371,3.052449e+05,1.049061,6.005862e+05
std,188.462426,0.820979,2.147768,3.157692,10.512396,1.433422e+06,1.912729e+07,3.927355e+06,1.977963e+07,2.669459e+07,4.912980e+06,4.544838e+06,2.335893e+07,2.713995e+09,1.153842e+08,0.621833,0.720198,0.725434,1.736454,1.785122,53.445528,5661.059297,5739.968774,436.670719,1964.226548,8796.472481,5390.915344,26930.742020,9666.183962,3124.491488,6049.678489,1044.597044,15.208396,0.683063,0.574002,10.621984,9.328810,7.969215,0.423802,45.085889,36.765769,0.412858,6.378946,0.196973,0.291146,0.335210,1.647757e+06,1.806160,5.842405e+06
min,2990.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,5.395430e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.255000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166600,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.021000,0.000000,0.000000,0.000000,0.048528,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00
25%,3358.000000,0.070788,0.000000,0.000000,0.000000,1.400400e+04,6.869740e+05,1.929440e+05,1.368502e+06,4.500730e+05,2.172565e+05,2.758010e+05,1.921660e+05,1.657646e+07,5.007636e+07,0.000000,0.000000,0.000000,0.000000,0.000000,40.305000,1068.667500,797.145000,14.865000,1231.500000,3375.000000,2068.000000,60.000000,9765.000000,2312.000000,6297.000000,194.000000,2.333400,0.000000,0.000000,6.000000,2.000000,4.000000,0.000000,136.191750,87.055500,0.000000,2.020000,0.411672,0.000000,0.000000,7.710079e+01,0.000000,4.560141e-02
50%,3484.000000,0.267575,0.062000,0.000000,0.000000,1.155330e+05,2.704328e+06,9.639420e+05,5.506295e+06,3.707559e+06,6.986440e+05,1.163110e+06,1.605792e+06,2.814143e+07,8.442428e+07,0.000000,0.000000,0.000000,0.000000,0.000000,89.167500,5976.225000,5548.935000,60.660000,2203.500000,8214.000000,5110.000000,486.000000,15057.000000,5658.000000,10584.000000,598.000000,7.833400,0.000000,0.000000,10.000000,6.000000,6.000000,0.000000,150.000000,118.441500,0.220000,5.050000,0.588648,0.000000,0.000000,3.181243e+02,0.000000,1.703996e-01
75%,3620.000000,0.719512,1.282000,0.000000,0.000000,5.259280e+05,6.993831e+06,3.751516e+06,1.694552

In [ ]:
train.describe()

,form_field1,form_field2,form_field3,form_field4,form_field5,form_field6,form_field7,form_field8,form_field9,form_field10,form_field11,form_field12,form_field13,form_field14,form_field15,form_field16,form_field17,form_field18,form_field19,form_field20,form_field21,form_field22,form_field23,form_field24,form_field25,form_field26,form_field27,form_field28,form_field29,form_field30,form_field31,form_field32,form_field33,form_field34,form_field35,form_field36,form_field37,form_field38,form_field39,form_field40,form_field41,form_field42,form_field43,form_field44,form_field45,form_field46,form_field48,form_field49,form_field50
count,53471.000000,52156.000000,55645.000000,55645.000000,55645.000000,4.264000e+04,5.083700e+04,4.264000e+04,4.799200e+04,5.564500e+04,2.457900e+04,4.610500e+04,5.011100e+04,5.600000e+04,3.352500e+04,42964.000000,44849.000000,45598.000000,55996.000000,55645.000000,40146.000000,35600.000000,27877.000000,42703.000000,50550.000000,48562.000000,46701.000000,55645.000000,55645.000000,30491.000000,16592.000000,50550.000000,54744.000000,55645.000000,32852.000000,54005.000000,50550.000000,55645.000000,51789.000000,12271.000000,17771.000000,54677.000000,55432.000000,50617.000000,24683.000000,40096.000000,3.511100e+04,55645.000000,4.494400e+04
mean,3491.795665,0.550737,1.052225,0.851979,1.956317,6.244479e+05,6.865210e+06,2.626690e+06,1.316002e+07,1.185585e+07,2.054715e+06,2.771113e+06,6.009944e+06,6.914875e+07,1.145041e+08,0.230425,0.265692,0.271832,0.948800,1.031593,86.500380,6628.316381,6452.875072,104.558516,2624.555935,10106.554899,6190.516948,17274.315141,16518.540695,5213.246663,10857.565755,938.345321,13.926583,0.230245,0.199866,12.859069,8.480435,8.601420,0.090849,147.797977,108.117363,0.368215,6.634511,0.563377,0.072520,0.095371,3.052449e+05,1.049061,6.005862e+05
std,188.462426,0.820979,2.147768,3.157692,10.512396,1.433422e+06,1.912729e+07,3.927355e+06,1.977963e+07,2.669459e+07,4.912980e+06,4.544838e+06,2.335893e+07,2.713995e+09,1.153842e+08,0.621833,0.720198,0.725434,1.736454,1.785122,53.445528,5661.059297,5739.968774,436.670719,1964.226548,8796.472481,5390.915344,26930.742020,9666.183962,3124.491488,6049.678489,1044.597044,15.208396,0.683063,0.574002,10.621984,9.328810,7.969215,0.423802,45.085889,36.765769,0.412858,6.378946,0.196973,0.291146,0.335210,1.647757e+06,1.806160,5.842405e+06
min,2990.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,5.395430e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.255000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166600,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.021000,0.000000,0.000000,0.000000,0.048528,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00
25%,3358.000000,0.070788,0.000000,0.000000,0.000000,1.400400e+04,6.869740e+05,1.929440e+05,1.368502e+06,4.500730e+05,2.172565e+05,2.758010e+05,1.921660e+05,1.657646e+07,5.007636e+07,0.000000,0.000000,0.000000,0.000000,0.000000,40.305000,1068.667500,797.145000,14.865000,1231.500000,3375.000000,2068.000000,60.000000,9765.000000,2312.000000,6297.000000,194.000000,2.333400,0.000000,0.000000,6.000000,2.000000,4.000000,0.000000,136.191750,87.055500,0.000000,2.020000,0.411672,0.000000,0.000000,7.710079e+01,0.000000,4.560141e-02
50%,3484.000000,0.267575,0.062000,0.000000,0.000000,1.155330e+05,2.704328e+06,9.639420e+05,5.506295e+06,3.707559e+06,6.986440e+05,1.163110e+06,1.605792e+06,2.814143e+07,8.442428e+07,0.000000,0.000000,0.000000,0.000000,0.000000,89.167500,5976.225000,5548.935000,60.660000,2203.500000,8214.000000,5110.000000,486.000000,15057.000000,5658.000000,10584.000000,598.000000,7.833400,0.000000,0.000000,10.000000,6.000000,6.000000,0.000000,150.000000,118.441500,0.220000,5.050000,0.588648,0.000000,0.000000,3.181243e+02,0.000000,1.703996e-01
75%,3620.000000,0.719512,1.282000,0.000000,0.000000,5.259280e+05,6.993831e+06,3.751516e+06,1.694552

# EDA Proper and Feature Engr

In [ ]:
data=pd.concat([train,test],sort=False).reset_index(drop=True)

In [ ]:
data.drop(['form_field48','form_field49'],inplace=True,axis=1)

In [ ]:

# Creating columns from missing values columns
def create_na_columns(x):
    if (pd.isnull(x)):
        return 1
    else: 
        return 0
features=test.columns.drop(['Applicant_ID','form_field47','form_field48','form_field49'])
for feature in features : 
    data[feature + "_na"]=data[feature].apply(create_na_columns)
    #test[feature + "_na"]=test[feature].apply(create_na_columns)    

# For best Model, Do not RUn

In [ ]:
# Creating Weekly Features
def create_week(x):
  if(pd.notnull(x)):
    return x/7
  else:
    pass
features= ['form_field28','form_field29','form_field30','form_field31']
for feature in features : 
    data[feature + "_week"]=data[feature].apply(create_week)

In [ ]:
# Creating Monthly Features
def create_month(x):
  if(pd.notnull(x)):
    return x/30
  else:
    pass
features= ['form_field28','form_field29','form_field30','form_field31']
for feature in features : 
    data[feature + "_month"]=data[feature].apply(create_month)

In [ ]:
# Creating Yearly Features
def create_year(x):
  if(pd.notnull(x)):
    return x/365
  else:
    pass
features= ['form_field28','form_field29','form_field30','form_field31']
for feature in features : 
    data[feature + "_year"]=data[feature].apply(create_year)

In [ ]:
# Creating an Extra Year Column for Form_field33
data['form_field33_year'] = data['form_field33'] / 12

In [ ]:
data.form_field47.replace({'charge':0, 'lending':1}, inplace=True)
data.default_status.replace({'no':0, 'yes':1}, inplace=True)

In [ ]:
train=data[data.default_status.notnull()].reset_index(drop=True)
test=data[data.default_status.isna()].reset_index(drop=True)

In [ ]:
train.shape, test.shape

((56000, 97), (24000, 97))

In [ ]:
train.shape, test.shape

((56000, 114), (24000, 114))

In [ ]:
train = train.fillna(-999)

In [ ]:
test.drop('default_status',axis=1, inplace=True)

In [ ]:
test = test.fillna(-999)

In [ ]:
test_id = test.Applicant_ID

In [ ]:
features = train.columns.drop(['Applicant_ID', 'default_status'])
X = pd.get_dummies(train[features])
y = train.default_status
test = pd.get_dummies(test[features])

# Categorcal features

In [ ]:
from catboost import CatBoostClassifier

In [ ]:
from catboost import CatBoostClassifier
errcb2=[]
y_pred_totcb2=[]
from sklearn.model_selection import KFold,StratifiedKFold, TimeSeriesSplit
from sklearn.metrics import plot_confusion_matrix, plot_roc_curve, roc_auc_score, auc, classification_report
fold=KFold(n_splits=20)
i=1
for train_index, test_index in fold.split(X,y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    m2  = CatBoostClassifier(n_estimators=10000,eval_metric='AUC',learning_rate=0.008, random_seed= seed, use_best_model=True)
    m2.fit(X_train,y_train,eval_set=[(X_train,y_train),(X_test, y_test)], early_stopping_rounds=250,verbose=100)#erly100
    preds=m2.predict_proba(X_test)
    print("AUC: ",roc_auc_score(y_test,preds[:, 1]))
    errcb2.append(roc_auc_score(y_test,preds[:, 1]))
    p2 = m2.predict_proba(test)[:, 1]
    y_pred_totcb2.append(p2)
np.mean(errcb2)

0:	test: 0.7882935	test1: 0.7886130	best: 0.7886130 (0)	total: 46.9ms	remaining: 11m 42s
100:	test: 0.8268309	test1: 0.8122994	best: 0.8122994 (100)	total: 4.84s	remaining: 11m 54s
200:	test: 0.8323937	test1: 0.8167372	best: 0.8167372 (200)	total: 9.58s	remaining: 11m 45s
300:	test: 0.8358177	test1: 0.8187407	best: 0.8187407 (300)	total: 14.3s	remaining: 11m 36s
400:	test: 0.8383381	test1: 0.8204528	best: 0.8204528 (400)	total: 19s	remaining: 11m 29s
500:	test: 0.8404326	test1: 0.8213886	best: 0.8213886 (500)	total: 23.5s	remaining: 11m 21s
600:	test: 0.8422460	test1: 0.8224473	best: 0.8224473 (600)	total: 28.1s	remaining: 11m 13s
700:	test: 0.8439692	test1: 0.8234330	best: 0.8234330 (700)	total: 32.7s	remaining: 11m 7s
800:	test: 0.8455913	test1: 0.8243330	best: 0.8243330 (800)	total: 37.3s	remaining: 11m
900:	test: 0.8469989	test1: 0.8250437	best: 0.8250586 (896)	total: 41.9s	remaining: 10m 55s
1000:	test: 0.8483684	test1: 0.8256189	best: 0.8256189 (1000)	total: 46.4s	remaining: 10m 

In [ ]:
np.mean(errcb2)  
#nsplit = 20

0.8410537106374985

In [ ]:
np.mean(errcb2)  
#nsplit = 20

0.8401239923251594

In [ ]:
np.mean(errcb2)  
#nsplit = 10

0.8396989213221833

In [ ]:
np.mean(errcb2)

0.8400188437373781

# Scaler Part

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
# standize the matrix for training data
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [ ]:
from catboost import CatBoostClassifier
errcb2=[]
y_pred_totcb2=[]
from sklearn.model_selection import KFold,StratifiedKFold, TimeSeriesSplit
from sklearn.metrics import plot_confusion_matrix, plot_roc_curve, roc_auc_score, auc, classification_report
fold=KFold(n_splits=20)
i=1
for train_index, test_index in fold.split(X,y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    m2  = CatBoostClassifier(n_estimators=10000,eval_metric='AUC',learning_rate=0.008, random_seed= seed, use_best_model=True)
    m2.fit(X_train,y_train,eval_set=[(X_train,y_train),(X_test, y_test)], early_stopping_rounds=250,verbose=100)#erly100
    preds=m2.predict_proba(X_test)
    print("AUC: ",roc_auc_score(y_test,preds[:, 1]))
    errcb2.append(roc_auc_score(y_test,preds[:, 1]))
    test_scaler = scaler.transform(test)
    p2 = m2.predict_proba(test_scaler)[:, 1]
    y_pred_totcb2.append(p2)
np.mean(errcb2)

0:	test: 0.7882935	test1: 0.7886130	best: 0.7886130 (0)	total: 47.4ms	remaining: 7m 53s
100:	test: 0.8271158	test1: 0.8119626	best: 0.8119850 (96)	total: 4.95s	remaining: 8m 5s
200:	test: 0.8326312	test1: 0.8165074	best: 0.8165074 (200)	total: 9.78s	remaining: 7m 56s
300:	test: 0.8360063	test1: 0.8188733	best: 0.8188733 (300)	total: 14.6s	remaining: 7m 49s
400:	test: 0.8384658	test1: 0.8205206	best: 0.8205288 (398)	total: 19.2s	remaining: 7m 39s
500:	test: 0.8404799	test1: 0.8216620	best: 0.8216925 (498)	total: 23.9s	remaining: 7m 32s
600:	test: 0.8423833	test1: 0.8229435	best: 0.8229435 (600)	total: 28.5s	remaining: 7m 25s
700:	test: 0.8440698	test1: 0.8237988	best: 0.8238591 (691)	total: 33.2s	remaining: 7m 20s
800:	test: 0.8456021	test1: 0.8245013	best: 0.8245013 (800)	total: 37.9s	remaining: 7m 15s
900:	test: 0.8470236	test1: 0.8250706	best: 0.8250706 (900)	total: 42.6s	remaining: 7m 10s
1000:	test: 0.8483146	test1: 0.8256249	best: 0.8256249 (1000)	total: 47.3s	remaining: 7m 5s
110

0.8408994650952103

In [ ]:
np.mean(errcb2)

0.8408994650952103

In [ ]:
np.mean(errcb2)

0.8399447933966073

In [ ]:
np.mean(errcb2)

0.8391763411977106

In [ ]:
np.mean(errcb2)

0.8394001437194116

In [ ]:
d = {'Applicant_ID' : test_id, 'default_status': np.mean(y_pred_totcb2,0)}

In [ ]:
sub = pd.DataFrame(data=d)
sub = sub[['Applicant_ID', 'default_status']]

In [ ]:
sub.to_csv("temmy__scal_drop_na_cat.csv",index  = False)